In [ ]:
# Notebook for testing

In [ ]:
import numpy as np
import pandas as pd
import recometrics
import implicit
from scipy.sparse import coo_matrix
import implicit.evaluation
from implicit.evaluation import mean_average_precision_at_k
from implicit.evaluation import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
data = [1,3,2,3,4,2,5,6,3,8,4,5,3,4,7,6,4,8,1,4,3,6,7,5,4,3,3,5,5,7,1,3,4,3,2]
rows = [0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5,6,6,6,6,6]
cols = [0,1,5,3,2,6,2,3,0,5,0,6,5,1,4,3,1,6,5,2,0,4,3,6,1,2,1,4,0,3,1,3,5,6,2]

In [ ]:
test_coo = coo_matrix((data, (rows, cols)), shape=(7, 7))

In [ ]:
t_train, t_test, t_users_test = \
    recometrics.split_reco_train_test(
        test_coo, split_type="joined",
        users_test_fraction = None,
        max_test_users = 2,
        items_test_fraction = 0.3
    )

In [ ]:
test_model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20, num_threads=4)
test_model.fit(20 * t_train)

In [ ]:
t_train_data = t_train[:t_test.shape[0]]
t_test_data = t_test
t_user_factors = test_model.user_factors[:t_test.shape[0]]
t_item_factors = test_model.item_factors
t_train_data.todense()

In [ ]:
recs = test_model.recommend(user_items=t_train_data, userid=list(range(0,t_train_data.shape[0])), filter_already_liked_items=True, N = 4)

MPF evaluation function testing:

In [ ]:
#for idx, u in enumerate(test_data.toarray()) :
zähler_liste = []
nenner_liste = []
for u in np.unique(t_test_data.tocoo().row) :
    #mask = (t_train_data[u].toarray()[0] == 0)
    #pred_temp = []
    #for i in t_item_factors :
    #    pred_temp.append(np.inner(t_user_factors[u], i)) # training beobachtungen auslassen. Pro user die trainings items aus train_data weglassen
    recs = test_model.recommend(user_items=t_train_data, userid=list(range(0,t_train_data.shape[0])), filter_already_liked_items=True, N = 4)
    pred_temp = recs[1][u]
    r = t_test_data[u].toarray()[0][recs[0][u]]
    df = pd.DataFrame({'r' : r, 'pred' : pred_temp})
    df = df.sort_values(by=['pred'], ascending=False)
    index = list(range(0,len(df)))
    df['rankui'] = [x / (len(df)-1) for x in index]
    zähler_liste.append(np.inner(df.r, df.rankui))
    nenner_liste.append(sum(df.r))
    mpr_list = [m/n for m, n in zip(zähler_liste, nenner_liste)]
    name = 'MPR@' + str(k)
    res = pd.DataFrame({name : mpr_list})
res

In [ ]:
# MPR function
def MPR(model, train_data, test_data, user_f, item_f, k) :
    zähler_liste = []
    nenner_liste = []
    #for idx, u in enumerate(test_data.toarray()) :
    for u in np.unique(test_data.tocoo().row) :
        #mask = (train_data[u].toarray()[0] == 0)
        #pred_temp = []
        #for i in item_f[mask] :
        #    pred_temp.append(np.inner(user_f[u], i)) # training beobachtungen auslassen. Pro user die trainings items aus train_data weglassen
        #df = pd.DataFrame({'r' : test_data[u].toarray()[0][mask], 'pred' : pred_temp}) #, 'train' : train_data[u].toarray()[0][mask]})
        recs = model.recommend(user_items=train_data, userid=list(range(0,train_data.shape[0])), filter_already_liked_items=True, N = k)
        pred_temp = recs[1][u]
        r = test_data[u].toarray()[0][recs[0][u]]
        r = np.clip(r, a_min=0, a_max=1)
        df = pd.DataFrame({'r' : r, 'pred' : pred_temp})
        df = df.sort_values(by=['pred'], ascending=False)
        #df = df.drop(df[df.train > 0].index)
        index = list(range(0,len(df)))
        df['rankui'] = [x / (len(df)-1) for x in index]
        #df = df.iloc[:k, ]
        zähler_liste.append(np.inner(df.r, df.rankui))
        if sum(df.r) != 0:
            nenner_liste.append(sum(df.r))
        else:
            nenner_liste.append(1)
        if u % 100 == 0:
            print(u)
    mpr_list = [m/n for m, n in zip(zähler_liste, nenner_liste)]
    name = 'MPR@' + str(k)
    return pd.DataFrame({name : mpr_list})
    

In [ ]:
MPR(test_model, t_train_data, t_test_data, t_user_factors, t_item_factors, 4)